# Third home assignment Social Data Science

# Temporal networks

Consider a human contact dataset (highschool_2011.csv) which consists of contacts between highschool students. Column 1 indicates the time and columns 2 and 3 indicates the node ids (you can ignore the rest of the columns). 

Write a function that takes as input the dataset and calculates the temporal correlation present in the dataset. 

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
file_path = '../dataset/highschool_2011.csv'
df_school = pd.read_csv(file_path, sep='\t', header=None)
df_school = df_school.iloc[:, :3]
df_school.columns = ['time', 'id1', 'id2']
# df_school = df_school.sort_values('time')

In [3]:
def calc_correlation(df_school, edge):
    value = 0
    for t_index in range(len(unique_time)-1):
        t_edges = df_school.loc[df_school['time'] == unique_time[t_index]].iloc[:, 1:].values
        t1_edges = df_school.loc[df_school['time'] == unique_time[t_index+1]].iloc[:, 1:].values
        if (edge in t_edges) & (edge in t1_edges):
            for j in unique_edges:
                count = 0
                edge_pair = [edge, j]
                if (edge_pair in t_edges) & (edge_pair in t1_edges):
                    count += 1
            value += count / np.sqrt(len(t_edges) * len(t1_edges))
    C_i = value / (len(unique_time) - 1)
    return C_i

In [4]:
def count_common_edges(list1, list2):
    count = 0
    for i in range(len(list1)):
        edge = list1[i]
        for j in range(len(list2)):
            next_edge = list2[j]
            # count common edge list
            if (edge == next_edge).all():
                count += 1
    return count * 2

In [5]:
def calc_temporal_corr(df_school):
    all_time = df_school.groupby('time')
    unique_edges = set(list(df_school['id1']) + list(df_school['id2']))
    N = len(unique_edges)
    T = len(df_school['time'].unique())
    
    C = 0
    a_t = 0
    a_t1 = 0
    t_edge = []
    for t, edges in all_time:
        t1_edge = edges.iloc[:, 1:].values
        a_t1 = len(t1_edge)
        common_edge = count_common_edges(t_edge, t1_edge)
        if common_edge != 0:
            C += common_edge / np.sqrt(a_t * a_t1)
        t_edge = t1_edge
        a_t = a_t1
    C = C / (N * (T-1))
    return C

In [6]:
temporal_corr = calc_temporal_corr(df_school)
print('Temporal Correlation: ', temporal_corr)

Temporal Correlation:  0.010399319199856231


Write a function to create a null model of the network by randomly shuffling the time stamps of the edges. Typically, consider a random pair of edges and change their time stamps (repeat this step 1000 times). Input to the function should be the network only. Recalculate the temporal correlation in this null model. 

In [7]:
def null_model(df_school):
    df_shuffle = df_school.copy()
    time_index = df_shuffle.index
    for i in range(10000):
        # random choose 2 index to swap
        swap_index = np.random.choice(time_index, 2)
        df_shuffle['time'][swap_index[0]] = df_school['time'][swap_index[1]]
        df_shuffle['time'][swap_index[1]] = df_school['time'][swap_index[0]]
    shuffle_corr = calc_temporal_corr(df_shuffle)
    return shuffle_corr

In [8]:
null_model_corr = null_model(df_school)
print('Temporal correlation for null mode: ', null_model_corr)

Temporal correlation for null mode:  0.003357173749101719


From the contact information provided in the dataset write a function to calculate the activity potential of each node i (F(i)). The function should take as input the network and return a dictionary of nodes and the corresponding activity potential. Now write a function to generate the network for next time step using the activity-driven network model. You can set the value of m (the number of links generated by each active node to 2). Note that a node i becomes active with a probability alpha\*F(i), alpha = 10

In [9]:
def calc_act_potential(df_school):
    df = df_school.copy()
    act_potential = {}
    
    edges = list(df['id1']) + list(df['id2'])
    edge_contact = Counter(edges)
    unique_edges = set(edges)
    all_contacts = len(df)

    for edge_num in unique_edges:
        act_potential[edge_num] = edge_contact[edge_num] / all_contacts
        
    return act_potential

In [10]:
def act_driven_model(df_school):
    act_potential = calc_act_potential(df_school)
    scale_act_potential = {}
    # scale activity potential probability
    for i in act_potential.keys():
        scale_act_potential[i] = act_potential[i] * 10
        
    active_nodes = []
    for i in scale_act_potential.keys():
        if scale_act_potential[i] > np.random.random(1):
            active_nodes.append(i)

    edges = list(scale_act_potential.keys())
    all_links = []
    for active_node in active_nodes:
        remain_nodes = [e for e in edges if e != active_node]
        # random choose 2 nodes to build a link
        neighbor_nodes = np.random.choice(remain_nodes, size=2)
        links = [(active_node, neighbor_nodes[0]), (active_node, neighbor_nodes[1])]
        all_links += links
    
    return all_links

In [11]:
# links generaged for next time step
links = act_driven_model(df_school)
links

[(2, 65),
 (2, 94),
 (22, 10),
 (22, 63),
 (28, 11),
 (28, 18),
 (41, 73),
 (41, 53),
 (43, 106),
 (43, 120),
 (49, 110),
 (49, 50),
 (54, 114),
 (54, 9),
 (56, 91),
 (56, 60),
 (75, 110),
 (75, 119),
 (85, 15),
 (85, 5),
 (89, 79),
 (89, 37),
 (98, 30),
 (98, 110),
 (99, 71),
 (99, 117),
 (100, 39),
 (100, 45),
 (103, 91),
 (103, 35),
 (106, 16),
 (106, 72)]

Write a function to obtain the ego (immediate neighbors) for each node. Your function should take as input the network and the node id and return its ego.
Using the activity potential calculated previously write a function to generate the network for the next time step using the activity-driven network model with memory. Any node links with a previously contacted node with probability n/n+1 (n is the size of its ego) or with a new node with probability 1/n+1. Note that a node i becomes active with a probability alpha\*F(i), alpha = 10

In [12]:
def find_ego(df_school):
    df = df_school.copy()
    edges = list(df['id1']) + list(df['id2'])
    unique_edges = set(edges)
    ego = {}
    for e in unique_edges:
        # find immediate neighbors
        e_neighbor = set(df.loc[df['id1'] == e, 'id2'].tolist() + df.loc[df['id2'] == e, 'id1'].tolist())
        ego[e] = e_neighbor
    return ego

In [13]:
ego = find_ego(df_school)
# ego

In [16]:
def act_driven_memory(df_school):
    
    act_potential = calc_act_potential(df_school)
    ego = find_ego(df_school)
    
    scale_act_potential = {}
    # scale activity potential probability
    for i in act_potential.keys():
        scale_act_potential[i] = act_potential[i] * 10

    active_nodes = []
    for i in scale_act_potential.keys():
        if scale_act_potential[i] > np.random.random(1):
            active_nodes.append(i)

    edges = list(scale_act_potential.keys())
    all_links = []
    for active_node in active_nodes:
        memory_node_prob = {}
        remain_nodes = [e for e in edges if e != active_node]
        n = len(ego[active_node])
        for e in remain_nodes:
            if e in ego[active_node]:
                memory_node_prob[e] = n / (n+1)
            else:
                memory_node_prob[e] = 1 / (n+1)
            
        p_list = np.array(list(memory_node_prob.values()))

        # standarize probability with activity driven with memory model
        memory_p = p_list / sum(p_list)
        neighbor_nodes = np.random.choice(remain_nodes, size=2, p=memory_p)
        memory_links = [(active_node, neighbor_nodes[0]), (active_node, neighbor_nodes[1])]
        all_links += memory_links
    
    return all_links


In [17]:
memory_links = act_driven_memory(df_school)
memory_links

[(8, 41),
 (8, 42),
 (11, 37),
 (11, 107),
 (22, 74),
 (22, 101),
 (27, 63),
 (27, 100),
 (28, 49),
 (28, 76),
 (29, 88),
 (29, 98),
 (37, 63),
 (37, 64),
 (39, 21),
 (39, 53),
 (42, 101),
 (42, 47),
 (43, 75),
 (43, 58),
 (56, 29),
 (56, 108),
 (57, 67),
 (57, 75),
 (59, 27),
 (59, 13),
 (66, 106),
 (66, 26),
 (77, 54),
 (77, 59),
 (82, 32),
 (82, 3),
 (85, 13),
 (85, 121),
 (90, 31),
 (90, 109),
 (102, 82),
 (102, 34),
 (103, 74),
 (103, 48),
 (106, 23),
 (106, 68),
 (116, 5),
 (116, 9)]